In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Carregar dados
data = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima.csv')

# Convertendo 'Data' para datetime e calculando a diferença diária do 'Preco_Dolar'
data['Data'] = pd.to_datetime(data['Data'])
data['Diff_Preco_Dolar'] = data['Preco_Dolar'].diff()
data = data.dropna()

# Tratamento de outliers (exemplo simples)
# Substitua com seu método preferido de tratamento de outliers
# data = seu_metodo_de_tratamento_de_outliers(data)

# Separando os dados para treino, validação e teste
cutoff_date = pd.to_datetime("2023-01-01")
train_val_data = data[data['Data'] < cutoff_date]
test_data = data[data['Data'] >= cutoff_date]

# Preparação dos dados
features_train_val = train_val_data.drop(['Preco_Dolar', 'Diff_Preco_Dolar', 'Data'], axis=1)
target_train_val = train_val_data['Diff_Preco_Dolar']

# Dividindo treino e validação
features_train, features_val, target_train, target_val = train_test_split(features_train_val, target_train_val, test_size=0.2, random_state=42)

# Ajuste fino dos hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(features_train, target_train)

best_params = grid_search.best_params_
print("Melhores hiperparâmetros:", best_params)

rf_best = RandomForestRegressor(**best_params, random_state=42)
rf_best.fit(features_train, target_train)

# Avaliação usando Gradient Boosting para comparação
gb = GradientBoostingRegressor(random_state=42)
gb.fit(features_train, target_train)

# Avaliação do modelo
window_size = 30
mse_values_rf, r2_values_rf = [], []
mse_values_gb, r2_values_gb = [], []
all_predictions_rf, all_predictions_gb = [], []
all_actual_values = []

for start in range(0, len(test_data) - window_size + 1):
    end = start + window_size
    test_subset = test_data.iloc[start:end]
    features_test = test_subset.drop(['Preco_Dolar', 'Diff_Preco_Dolar', 'Data'], axis=1)
    target_test = test_subset['Diff_Preco_Dolar']

    prediction_rf = rf_best.predict(features_test)
    prediction_gb = gb.predict(features_test)

    mse_values_rf.append(mean_squared_error(target_test, prediction_rf))
    r2_values_rf.append(r2_score(target_test, prediction_rf))

    mse_values_gb.append(mean_squared_error(target_test, prediction_gb))
    r2_values_gb.append(r2_score(target_test, prediction_gb))

    all_predictions_rf.extend(prediction_rf)
    all_predictions_gb.extend(prediction_gb)
    all_actual_values.extend(target_test)

# Exibindo os resultados para Random Forest
print("Random Forest - Média do MSE:", np.mean(mse_values_rf))
print("Random Forest - Média do R²:", np.mean(r2_values_rf))

# Exibindo os resultados para Gradient Boosting
print("Gradient Boosting - Média do MSE:", np.mean(mse_values_gb))
print("Gradient Boosting - Média do R²:", np.mean(r2_values_gb))


Fitting 3 folds for each of 144 candidates, totalling 432 fits


c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Melhores hiperparâmetros: {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Random Forest - Média do MSE: 9.146935874181615
Random Forest - Média do R²: -0.02339024755003896
Gradient Boosting - Média do MSE: 10.252676417009145
Gradient Boosting - Média do R²: -0.2633344220592247


In [26]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima.csv')
data_dolar = pd.read_csv(file_path_dolar)
data_dolar.fillna(0, inplace=True)
# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.sort_values(by='Data')

# Calcular a diferença diária do 'Preco_Dolar'
data_dolar['Dif_Preco_Dolar'] = data_dolar['Preco_Dolar'].diff()

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Dividir os dados em treino, teste e validação
# Filtrar os dados para treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Dif_Preco_Dolar'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Dif_Preco_Dolar'])

# O restante do código para tratamento de dados não numéricos, treinamento e avaliação do modelo permanece o mesmo.
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
#Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
model = LinearRegression()

model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())

MSE: 9.595039206728583
MAE: 2.260820458170779
R^2: 0.18082554425127229
           Data  Valor_Real  Valor_Previsto
5524 2020-01-02       -3.67       -0.407749
5525 2020-01-03       -1.07       -0.872377
5526 2020-01-06       -5.45       -0.020286
5527 2020-01-07       -0.83        0.177764
5528 2020-01-08       -1.42        0.415451


C:\Users\milen\AppData\Local\Temp\ipykernel_24468\3619221553.py:38: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
C:\Users\milen\AppData\Local\Temp\ipykernel_24468\3619221553.py:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')


In [48]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/teste.csv')
data_dolar = pd.read_csv(file_path_dolar)
data_dolar.fillna(0, inplace=True)
# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
data_dolar = data_dolar.sort_values(by='Data')

# Calcular a diferença diária do 'Preco_Dolar'
#data_dolar['Dif_Preco_Dolar'] = data_dolar['Preco_Dolar'].diff()

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Dividir os dados em treino, teste e validação
# Filtrar os dados para treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Dif_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Dif_Preco_Dolar','Pontos'])

y_test = test_data['Dif_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Dif_Preco_Dolar','Pontos'])

# O restante do código para tratamento de dados não numéricos, treinamento e avaliação do modelo permanece o mesmo.
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
#Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
model = LinearRegression()

model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())

MSE: 9.602041201664678
MAE: 2.262701273054817
R^2: 0.1802277504051658
           Data  Valor_Real  Valor_Previsto
5524 2020-01-02       -3.67       -0.400155
5525 2020-01-03       -1.07       -0.854128
5526 2020-01-06       -5.45       -0.011394
5527 2020-01-07       -0.83        0.065152
5528 2020-01-08       -1.42        0.429875


In [53]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import lightgbm as lgb

# Carregar os dados
file_path_dolar =('C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima.csv')
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime e ordenar os dados
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])
# data_dolar = data_dolar.sort_values(by='Data')

# Calcular a diferença diária do 'Preco_Dolar'
data_dolar['Diff_Preco_Dolar'] = data_dolar['Preco_Dolar'].diff()

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino, teste e validação
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Diff_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Diff_Preco_Dolar', 'TaxaSelic','OperacoesSeleic','UltimoEUR'])

y_test = test_data['Diff_Preco_Dolar']
X_test = test_data.drop(columns=['Data','Pontos', 'Diff_Preco_Dolar','TaxaSelic','OperacoesSeleic','UltimoEUR'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Treinar o modelo de regressão linear
model = LinearRegression()
#model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())




MSE: 3.86678536313008
MAE: 1.5224658329968275
R^2: 0.14074155332226146
           Data  Valor_Real  Valor_Previsto
5524 2020-01-02       -3.67       -0.573965
5527 2020-01-07       -0.83        0.192065
5529 2020-01-09       -2.08       -0.271280
5530 2020-01-10        1.16        0.207927
5531 2020-01-13       -3.78       -0.743161


C:\Users\milen\AppData\Local\Temp\ipykernel_24468\1562008122.py:48: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values_train = X_train.mean()
C:\Users\milen\AppData\Local\Temp\ipykernel_24468\1562008122.py:51: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values_test = X_test.mean()
C:\Users\milen\AppData\Local\Temp\ipykernel_24468\1562008122.py:57: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as lit

In [54]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

# Carregar os dados
file_path_dolar = 'C:/Users/milen/OneDrive/Documentos/TCC/new/sem_clima.csv'
data_dolar = pd.read_csv(file_path_dolar)

# Converter a coluna 'Data' para datetime
data_dolar['Data'] = pd.to_datetime(data_dolar['Data'])

# Calcular a diferença diária do 'Preco_Dolar'
data_dolar['Diff_Preco_Dolar'] = data_dolar['Preco_Dolar'].diff()

# Remover a primeira linha
data_dolar = data_dolar.iloc[1:]

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Aplicar a remoção de outliers
numeric_columns = data_dolar.select_dtypes(include=[np.number]).columns.tolist()
data_dolar = remove_outliers(data_dolar, numeric_columns)

# Dividir os dados em treino e teste
train_data = data_dolar[data_dolar['Data'] < pd.to_datetime("2020-01-01")]
test_data = data_dolar[data_dolar['Data'] >= pd.to_datetime("2020-01-01")]

# Selecionar variáveis dependentes e independentes
y_train = train_data['Diff_Preco_Dolar']
X_train = train_data.drop(columns=['Data', 'Pontos', 'Diff_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR'])

y_test = test_data['Diff_Preco_Dolar']
X_test = test_data.drop(columns=['Data', 'Pontos', 'Diff_Preco_Dolar', 'TaxaSelic', 'OperacoesSeleic', 'UltimoEUR'])

# Calcular a média para as colunas e substituir NaN pela média
mean_values_train = X_train.mean()
X_train.fillna(mean_values_train, inplace=True)

mean_values_test = X_test.mean()
X_test.fillna(mean_values_test, inplace=True)

# Tratar valores não numéricos
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
for col in non_numeric_columns:
    X_train[col] = pd.to_numeric(X_train[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col].str.replace('.', '').str.replace(',', '.'), errors='coerce')

# Definir hiperparâmetros a serem testados
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

# Inicializar o modelo LGBM
lgb_model = lgb.LGBMRegressor()

# Inicializar a pesquisa em grade com validação cruzada
grid_search = GridSearchCV(estimator=lgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# Executar a pesquisa em grade
grid_search.fit(X_train, y_train)

# Exibir os melhores hiperparâmetros encontrados
best_params = grid_search.best_params_
print("Melhores Hiperparâmetros:")
print(best_params)

# Treinar o modelo LGBM com os melhores hiperparâmetros
best_lgb_model = lgb.LGBMRegressor(**best_params)
best_lgb_model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = best_lgb_model.predict(X_test)

# Calcular as métricas de desempenho
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Exibir os resultados
print("MSE:", mse)
print("MAE:", mae)
print("R^2:", r2)

# Criar um DataFrame com as datas, valores reais e previstos
resultados = pd.DataFrame()
resultados['Data'] = test_data['Data']
resultados['Valor_Real'] = y_test
resultados['Valor_Previsto'] = y_pred

# Exibir os primeiros registros para verificar
print(resultados.head())


C:\Users\milen\AppData\Local\Temp\ipykernel_24468\343449117.py:48: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values_train = X_train.mean()
C:\Users\milen\AppData\Local\Temp\ipykernel_24468\343449117.py:51: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  mean_values_test = X_test.mean()
C:\Users\milen\AppData\Local\Temp\ipykernel_24468\343449117.py:57: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as litera

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000388 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4114
[LightGBM] [Info] Number of data points in the train set: 1816, number of used features: 19
[LightGBM] [Info] Start training from score 0.009312
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 